In [1]:
from pdfminer.high_level import extract_text
import os
import pandas as pd
import re
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import faiss
import numpy as np

model = SentenceTransformer('jhgan/ko-sbert-sts', use_auth_token=False)

c:\Users\wisj9\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\wisj9\AppData\Local\Programs\Python\Python312\Lib\site-packages\sentence_transformers\SentenceTransformer.py:195: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v4 of SentenceTransformers.
  warnings.warn(


## PDF 처리

In [13]:
# pdfs = [f"datas/건설안전지침/{i}" for i in os.listdir("datas/건설안전지침")]
# print("\n".join(pdfs))
# print(f"PDF 총 개수: {len(pdfs)}")

# words = {}
# for i in range(len(pdfs)):
#     pdf = extract_text(pdfs[i])

#     # pdf = pdf.replace("\n", " ")
#     pdf = pdf.replace("  ", " ")

#     pdf = pdf.replace('3. 용어의 정의 ·', "<delete>")
#     pdf = pdf.replace('3. 용어의 정의', "<용어 정리 시작>")
#     pdf = pdf.replace("3. 정의", "<용어 정리 시작>")
#     pdf = pdf.replace('3 용어의 정의', "<용어 정리 시작>")
#     pdf = pdf.replace("3. 용어의정의", "<용어 정리 시작>")

#     # pdf = pdf.replace("기타 이 지침에서", "<용어 정리 마감>")
#     # pdf = pdf.replace("그 밖의 용어의", "<용어 정리 마감>")
#     # pdf = pdf.replace("그 밖의 이 지침에", "<용어 정리 마감>")
#     pdf = pdf.replace("(2)", "<용어 정리 마감>")

#     pdf = pdf.split("<용어 정리 시작>")[1]
#     pdf = pdf.split("<용어 정리 마감>")
#     pdf = " ".join(pdf[1:])
#     print(pdf)
#     break


In [14]:
train = pd.read_csv("datas/train.csv")
test = pd.read_csv("datas/test.csv")
print(len(train), len(test))

23422 964


In [15]:
train.iloc[0]
test.iloc[0]

ID                                                  TEST_000
발생일시                                     2024-06-03 오전 09:39
사고인지 시간                                               정규작업 -
날씨                                                        맑음
기온                                                       27℃
습도                                                       53%
공사종류                                    건축 / 건축물 / 교정 및 군사시설
연면적                                                1,990.32㎡
층 정보                                            지상 1층, 지하 0층
인적사고                                                     부딪힘
물적사고                                                      전도
공종                                             건축 > 철근콘크리트공사
사고객체                                           건설기계 > 콘크리트펌프
작업프로세스                                                  타설작업
장소                                            교정 및 군사시설 / 외부
부위                                               콘크리트펌프 / 바닥
사고원인       펌프카 아웃트리거 바닥 

In [40]:
train["for_rag"] = train.apply(lambda row: f"""발생 일시: {row["발생일시"][:4]+"년"+row["발생일시"][5:7]+"월"+row["발생일시"][8:10]+"일"}({row["사고인지 시간"]})
날씨: {row["날씨"]}
기온/습도: {row["기온"]}/{row["습도"]}
공사 종류: {row["공사종류"]} / {row["공종"]} / {row["작업프로세스"]}
사고 내용(인명 사고): {row["인적사고"]}
사고 내용(재산 피해): {row["물적사고"]}
사고 발생 장소: {row["장소"]}
사고 원인: {row["사고원인"]}
""",axis = 1)

test["for_rag"] = test.apply(lambda row: f"""발생 일시: {row["발생일시"][:4]+"년"+row["발생일시"][5:7]+"월"+row["발생일시"][8:10]+"일"}({row["사고인지 시간"]})
날씨: {row["날씨"]}
기온/습도: {row["기온"]}/{row["습도"]}
공사 종류: {row["공사종류"]} / {row["공종"]} / {row["작업프로세스"]}
사고 내용(인명 사고): {row["인적사고"]}
사고 내용(재산 피해): {row["물적사고"]}
사고 발생 장소: {row["장소"]}
사고 원인: {row["사고원인"]}
""",axis = 1)

train["for_rag_weather"] = train.apply(lambda row: f"""발생 일시: {row["발생일시"][:4]+"년"+row["발생일시"][5:7]+"월"+row["발생일시"][8:10]+"일"}({row["사고인지 시간"]})
날씨: {row["날씨"]}
기온/습도: {row["기온"]}/{row["습도"]}
사고 원인: {row["사고원인"]}
""",axis = 1)

test["for_rag_weather"] = test.apply(lambda row: f"""발생 일시: {row["발생일시"][:4]+"년"+row["발생일시"][5:7]+"월"+row["발생일시"][8:10]+"일"}({row["사고인지 시간"]})
날씨: {row["날씨"]}
기온/습도: {row["기온"]}/{row["습도"]}
사고 원인: {row["사고원인"]}
""",axis = 1)

train["for_rag_accident"] = train.apply(lambda row: f"""
공사 종류: {row["공사종류"]} / {row["공종"]} / {row["작업프로세스"]}
사고 내용(인명 사고): {row["인적사고"]}
사고 내용(재산 피해): {row["물적사고"]}
사고 발생 장소: {row["장소"]}
사고 원인: {row["사고원인"]}
""",axis = 1)

test["for_rag_accident"] = test.apply(lambda row: f"""
사고 내용(인명 사고): {row["인적사고"]}
사고 내용(재산 피해): {row["물적사고"]}
사고 원인: {row["사고원인"]}
""",axis = 1)

try:
    doc_embeddings_weather = np.load("doc_embeddings_weather.npy")
except:
    doc_embeddings_weather = model.encode(list(train["for_rag_weather"]), convert_to_numpy=True)
    np.save("doc_embeddings_weather.npy", doc_embeddings_weather)

dim_weather = doc_embeddings_weather.shape[1]
index_weather = faiss.IndexFlatL2(dim_weather)
index_weather.add(doc_embeddings_weather)

try:
    doc_embeddings_accident = np.load("doc_embeddings_accident.npy")
except:
    doc_embeddings_accident = model.encode(list(train["for_rag_accident"]), convert_to_numpy=True)
    np.save("doc_embeddings_accident.npy", doc_embeddings_accident)

dim_accident = doc_embeddings_accident.shape[1]
index_accident = faiss.IndexFlatL2(dim_accident)
index_accident.add(doc_embeddings_accident)

try:
    doc_embeddings = np.load("doc_embeddings.npy")
except:
    doc_embeddings = model.encode(list(train["for_rag"]), convert_to_numpy=True)
    np.save("doc_embeddings.npy", doc_embeddings)

dim = doc_embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(doc_embeddings)

In [42]:
print("원문 쿼리")
query = test.iloc[0]["for_rag"]
query_weather = test.iloc[0]["for_rag_weather"]
query_accident = test.iloc[0]["for_rag_accident"]
print(query+"\n---------------------------------------------------------------------------------")
query_embedding = model.encode([query], convert_to_numpy=True)
query_embedding_weather = model.encode([query_weather], convert_to_numpy=True)
query_embedding_accident = model.encode([query_accident], convert_to_numpy=True)

# 가장 유사한 top_k 문서 검색
top_k = 2
distances, indices = index.search(query_embedding, top_k)
distances_weather, indices_weather = index_weather.search(query_embedding_weather, top_k)
distances_accident, indices_accident = index_accident.search(query_embedding_accident, top_k)

# 검색된 문서 출력
retrieved_docs = [list(train["for_rag"])[i] + list(train["재발방지대책 및 향후조치계획"])[i] for i in indices[0]]
retrieved_docs_weather = [list(train["for_rag"])[i] + list(train["재발방지대책 및 향후조치계획"])[i] for i in indices_weather[0]]
retrieved_docs_accident = [list(train["for_rag"])[i] + list(train["재발방지대책 및 향후조치계획"])[i] for i in indices_accident[0]]

for doc in retrieved_docs:
    print(doc+"\n")
print("-----------------------------------------------------------------")
for doc in retrieved_docs_weather:
    print(doc+"\n")
print("-----------------------------------------------------------------")
for doc in retrieved_docs_accident:
    print(doc+"\n")
print("-----------------------------------------------------------------")

원문 쿼리
발생 일시: 2024년06월03일(정규작업 -)
날씨: 맑음
기온/습도: 27℃/53%
공사 종류: 건축 / 건축물 / 교정 및 군사시설 / 건축 > 철근콘크리트공사 / 타설작업
사고 내용(인명 사고): 부딪힘
사고 내용(재산 피해): 전도
사고 발생 장소: 교정 및 군사시설 / 외부
사고 원인: 펌프카 아웃트리거 바닥 고임목을 3단으로 보강 했음에도, 지반 침하(아웃트리거 우측 상부 1개소)가 발생하였고,  좌, 우측 아웃트리거의 펼친 길이가 상이하고 타설 위치가 건물 끝부분 모서리에 위치하여 붐대호스를 최대로 펼치다 보니 장비에 대한 무게중심이 한쪽으로 쏠려 일부 전도되는 사고가 발생된 것으로 판단됨

---------------------------------------------------------------------------------
발생 일시: 2023년03월21일(정규작업 -)
날씨: 흐림
기온/습도: 10℃/53%
공사 종류: 건축 / 건축물 / 창고시설 / 건축 > 철근콘크리트공사 / 해체작업
사고 내용(인명 사고): 끼임
사고 내용(재산 피해): 없음
사고 발생 장소: 창고시설 / 내부
사고 원인: 1. 작업자의 불안전한 행동 : 해치발판인 경우 개폐구를 하부로 두고 받아내리기 했어야함2. 작업 전 T.B.M 활동 미흡 : 개폐자재 취급에 대한 교육 미흡
해체 작업 등 위험 작업 시 관리자 참여와 T.B.M 진행 및 작업 감독 실시와 시스템비계 작업팀의 불안전한 행동에 대한 특별안전보건교육 실시.

발생 일시: 2023년05월13일(정규작업 -)
날씨: 맑음
기온/습도: 27℃/60%
공사 종류: 건축 / 건축물 / 공동주택 / 건축 > 철근콘크리트공사 / 설치작업
사고 내용(인명 사고): 떨어짐(2미터 이상 ~ 3미터 미만)
사고 내용(재산 피해): 없음
사고 발생 장소: 공동주택 / 지하주차장 상부
사고 원인: 데크플레이트 개별 판재를 보 거푸집 상부에 설치하고자 데크플레이트 판재 야적부 상부로 이동하여 데크플레이트 자재